Convenience functions for automatic reloading .py files

In [1]:
%load_ext autoreload
%autoreload 2

Check whether video procesing in opencv is working (I had some trouble with it and had to build from source)

In [2]:
import os
#VIDEO_PATH = os.path.abspath('input/SanJoseBike.mp4')
VIDEO_PATH = os.path.abspath('input/SanJoseBike.avi')

COCO_WEIGHTS_PATH = os.path.abspath('mask_rcnn_coco.h5')
LOGS_DIR = os.path.abspath('logs')

BATCH_SIZE = 4
FPS = 30
INPUT_RES = (1920,1080)

In [3]:
import cv2
print(cv2.__version__)
video = cv2.VideoCapture(VIDEO_PATH)
print(video.isOpened())

3.4.4
True


In [4]:
from coco import coco
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = BATCH_SIZE

config = InferenceConfig()


Using TensorFlow backend.


In [5]:
import mrcnn
# Create model object in inference mode.
model = mrcnn.model.MaskRCNN(mode="inference", model_dir=LOGS_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_WEIGHTS_PATH, by_name=True)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [6]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [7]:
import numpy as np
def rotate_bound(image, angle):
    #Source: https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/
    
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

Get and process the frame from the video

In [ ]:
# Run detection
from mrcnn import visualize2
import time
i = 0

_, frame = video.read()
out = cv2.VideoWriter('output/output-{}.avi'.format(int(time.time())),cv2.VideoWriter_fourcc(*'DIVX'), FPS, (frame.shape[1],frame.shape[0]))


for _ in range(300):
    
    print(i)
    i+=1
    
    frames = []
    
    for batch_idx in range(BATCH_SIZE):
        
        success, frame = video.read()
        if not success: break
            
        #convert from opencv bgr to rgb
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #rotate (I don't know why but the videos from berkeley dataset were rotated)
        frames.append(rotate_bound(frame, 0))
        #frames.append(frame)
        
    results = model.detect(frames, verbose=False)
    
    for batch_idx in range(BATCH_SIZE):
        r = results[batch_idx]
                            
        out_img = visualize2.get_instances_image(frames[batch_idx], r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'])
    
        #height, width, layers = out_img.shape
        #size = (width,height)

        out.write(out_img)
    
out.release()

    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54

*** No instances to display *** 

55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
